In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame


# binning操作

In [2]:
years = [1990,1991,1992,2008,2012,2015,1987,1969,2013,2008,1999]

In [3]:
decade_bins = [1960,1970,1980,1990,2000,2010,2020]

In [4]:
decade_cat = pd.cut(years,decade_bins)
decade_cat

[(1980, 1990], (1990, 2000], (1990, 2000], (2000, 2010], (2010, 2020], ..., (1980, 1990], (1960, 1970], (2010, 2020], (2000, 2010], (1990, 2000]]
Length: 11
Categories (6, interval[int64]): [(1960, 1970] < (1970, 1980] < (1980, 1990] < (1990, 2000] < (2000, 2010] < (2010, 2020]]

In [5]:
decade_cat.codes

array([2, 3, 3, 4, 5, 5, 2, 0, 5, 4, 3], dtype=int8)

In [6]:
decade_cat.categories

IntervalIndex([(1960, 1970], (1970, 1980], (1980, 1990], (1990, 2000], (2000, 2010], (2010, 2020]],
              closed='right',
              dtype='interval[int64]')

In [7]:
pd.value_counts(decade_cat)

(2010, 2020]    3
(1990, 2000]    3
(2000, 2010]    2
(1980, 1990]    2
(1960, 1970]    1
(1970, 1980]    0
dtype: int64

In [8]:
decade_cat_2 = pd.cut(years,2,precision=1)   #等分成2个binning

In [9]:
decade_cat_2.codes

array([0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1], dtype=int8)

In [10]:
decade_cat_2.categories

IntervalIndex([(1969.0, 1992.0], (1992.0, 2015.0]],
              closed='right',
              dtype='interval[float64]')

In [11]:
pd.value_counts(decade_cat_2)

(1992.0, 2015.0]    6
(1969.0, 1992.0]    5
dtype: int64

# Groupby操作

## Series groupby操作

In [13]:
animals = DataFrame(np.arange(16).reshape(4,4),
                   columns=['W','X','Y','Z'],
                   index = ['Dog','Cat','House','Rabbit'])
print(animals)
animals.loc[1:2,['W','Y']] =  np.nan
animals

         W   X   Y   Z
Dog      0   1   2   3
Cat      4   5   6   7
House    8   9  10  11
Rabbit  12  13  14  15


,W,X,Y,Z
Dog,0.0,1,2.0,3
Cat,NaN,5,NaN,7
House,8.0,9,10.0,11
Rabbit,12.0,13,14.0,15


In [14]:
behavior_map = {'W':'good','X':'bad','Y':'good','Z':'bad'}
animal_col = animals.groupby(behavior_map,axis=1)

In [15]:
animal_col.sum()

,bad,good
Dog,4.0,2.0
Cat,12.0,0.0
House,20.0,18.0
Rabbit,28.0,26.0


In [16]:
behav_series = Series(behavior_map)
behav_series

W    good
X     bad
Y    good
Z     bad
dtype: object

In [17]:
animals.groupby(behav_series,axis=1).count()

,bad,good
Dog,2,2
Cat,2,0
House,2,2
Rabbit,2,2


In [19]:
animals.groupby(len).sum()

,W,X,Y,Z
3,0.0,6,2.0,10
5,8.0,9,10.0,11
6,12.0,13,14.0,15


In [20]:
keys = ['A','B','B','A']
animals.groupby([len,keys]).max()

W   X     Y   Z
3 A   0.0   1   2.0   3
  B   NaN   5   NaN   7
5 B   8.0   9  10.0  11
6 A  12.0  13  14.0  15

In [21]:
hier_col=pd.MultiIndex.from_arrays([['NY','NY','NY','SF','SF'],[1,2,3,1,2]],names=['City','sub_value'])
hier_col

MultiIndex(levels=[['NY', 'SF'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1], [0, 1, 2, 0, 1]],
           names=['City', 'sub_value'])

In [22]:
dframe_hr = DataFrame(np.arange(25).reshape(5,5),columns=hier_col)
dframe_hr = dframe_hr*100
dframe_hr

City         NY                SF      
sub_value     1     2     3     1     2
0             0   100   200   300   400
1           500   600   700   800   900
2          1000  1100  1200  1300  1400
3          1500  1600  1700  1800  1900
4          2000  2100  2200  2300  2400

## DataFrame groupby操作

In [24]:
df = DataFrame({'k1':['X','X','Y','Y','Z'],
               'k2':['alpha','beta','alpha','beta','alpha'],
               'dataset1':np.random.randn(5),
               'dataset2':np.random.randn(5)},columns=['k1','k2','dataset1','dataset2'])
df

,k1,k2,dataset1,dataset2
0,X,alpha,-0.360381,-2.217438
1,X,beta,-0.074364,-0.090229
2,Y,alpha,1.333226,-0.359792
3,Y,beta,0.277917,0.237005
4,Z,alpha,-0.398893,-1.458744


In [26]:
group1 = df['dataset1'].groupby(df['k1'])
group1.mean()

k1
X   -0.217372
Y    0.805572
Z   -0.398893
Name: dataset1, dtype: float64

In [27]:
cities = np.array(['NY','LA','LA','NY','NY'])
month = np.array(['Oct','Jun','Jan','Feb','Sep'])


In [28]:
df['dataset1'].groupby([cities,month]).mean()

LA  Jan    1.333226
    Jun   -0.074364
NY  Feb    0.277917
    Oct   -0.360381
    Sep   -0.398893
Name: dataset1, dtype: float64

In [30]:
df.groupby('k1').mean()

,dataset1,dataset2
k1,,
X,-0.217372,-1.153833
Y,0.805572,-0.061393
Z,-0.398893,-1.458744


In [31]:
df.groupby('k2').mean()

,dataset1,dataset2
k2,,
alpha,0.191317,-1.345324
beta,0.101777,0.073388


In [32]:
df.groupby(['k1','k2']).mean()

dataset1  dataset2
k1 k2                       
X  alpha -0.360381 -2.217438
   beta  -0.074364 -0.090229
Y  alpha  1.333226 -0.359792
   beta   0.277917  0.237005
Z  alpha -0.398893 -1.458744

# 用groupby遍历

In [36]:
print(df)
group1 = df['dataset1'].groupby(df['k1'])

  k1     k2  dataset1  dataset2
0  X  alpha -0.360381 -2.217438
1  X   beta -0.074364 -0.090229
2  Y  alpha  1.333226 -0.359792
3  Y   beta  0.277917  0.237005
4  Z  alpha -0.398893 -1.458744


In [35]:
for g in group1 : 
    print('NNNNN')
    print(g[0],g[1])

NNNNN
X 0   -0.360381
1   -0.074364
Name: dataset1, dtype: float64
NNNNN
Y 2    1.333226
3    0.277917
Name: dataset1, dtype: float64
NNNNN
Z 4   -0.398893
Name: dataset1, dtype: float64


In [37]:
group1.mean()

k1
X   -0.217372
Y    0.805572
Z   -0.398893
Name: dataset1, dtype: float64

In [40]:
for group_name,group in df.groupby('k1'):
    print("this is the %s group"%group_name)
    print(group)
    print('\n')
    

this is the X group
  k1     k2  dataset1  dataset2
0  X  alpha -0.360381 -2.217438
1  X   beta -0.074364 -0.090229


this is the Y group
  k1     k2  dataset1  dataset2
2  Y  alpha  1.333226 -0.359792
3  Y   beta  0.277917  0.237005


this is the Z group
  k1     k2  dataset1  dataset2
4  Z  alpha -0.398893 -1.458744




In [42]:
for (k1,k2) ,group in df.groupby(['k1','k2']):
    print('Key1 = %s key2 = %s'%(k1,k2))
    print(group)
    print('\n')

Key1 = X key2 = alpha
  k1     k2  dataset1  dataset2
0  X  alpha -0.360381 -2.217438


Key1 = X key2 = beta
  k1    k2  dataset1  dataset2
1  X  beta -0.074364 -0.090229


Key1 = Y key2 = alpha
  k1     k2  dataset1  dataset2
2  Y  alpha  1.333226 -0.359792


Key1 = Y key2 = beta
  k1    k2  dataset1  dataset2
3  Y  beta  0.277917  0.237005


Key1 = Z key2 = alpha
  k1     k2  dataset1  dataset2
4  Z  alpha -0.398893 -1.458744




In [43]:
group_dic = dict(list(df.groupby('k1')))

group_dic['X']

,k1,k2,dataset1,dataset2
0,X,alpha,-0.360381,-2.217438
1,X,beta,-0.074364,-0.090229


In [44]:
group_dic

{'X':   k1     k2  dataset1  dataset2
 0  X  alpha -0.360381 -2.217438
 1  X   beta -0.074364 -0.090229, 'Y':   k1     k2  dataset1  dataset2
 2  Y  alpha  1.333226 -0.359792
 3  Y   beta  0.277917  0.237005, 'Z':   k1     k2  dataset1  dataset2
 4  Z  alpha -0.398893 -1.458744}

# merge操作

In [46]:
df = DataFrame({'key':['X','Y','Z','X','X','Y'],'data':[0,1,2,3,4,5]})
df

,data,key
0,0,X
1,1,Y
2,2,Z
3,3,X
4,4,X
5,5,Y


In [47]:
df2 = DataFrame({'key':['W','Y','Z'],'data2':[0,1,2]})
df2

,data2,key
0,0,W
1,1,Y
2,2,Z


In [51]:
pd.merge(df,df2)

,data,key,data2
0,1,Y,1
1,5,Y,1
2,2,Z,2


In [52]:
pd.merge(df,df2,on='key')

,data,key,data2
0,1,Y,1
1,5,Y,1
2,2,Z,2


In [55]:
pd.merge(df,df2,on='key',how='inner')

,data,key,data2
0,1,Y,1
1,5,Y,1
2,2,Z,2


In [56]:
pd.merge(df,df2,on='key',how='left')

,data,key,data2
0,0,X,NaN
1,1,Y,1.0
2,2,Z,2.0
3,3,X,NaN
4,4,X,NaN
5,5,Y,1.0


In [57]:
pd.merge(df,df2,on='key',how='right')

,data,key,data2
0,1.0,Y,1
1,5.0,Y,1
2,2.0,Z,2
3,NaN,W,0


In [58]:
pd.merge(df,df2,on='key',how='outer')

,data,key,data2
0,0.0,X,NaN
1,3.0,X,NaN
2,4.0,X,NaN
3,1.0,Y,1.0
4,5.0,Y,1.0
5,2.0,Z,2.0
6,NaN,W,0.0


# merge index

In [60]:
df_left1 = DataFrame({'key':['X','Y','Z','X','Y'],'data_l':range(5)})
df_right1 = DataFrame({'data_r':[10,20]},index=['X','Y'])
df_left1

,data_l,key
0,0,X
1,1,Y
2,2,Z
3,3,X
4,4,Y


In [61]:
df_right1

,data_r
X,10
Y,20


In [62]:
pd.merge(df_left1,df_right1,left_on='key',right_index=True)

,data_l,key,data_r
0,0,X,10
3,3,X,10
1,1,Y,20
4,4,Y,20


# Outliers

In [63]:
np.random.seed(2018)
df = DataFrame(np.random.randn(1000,4))


In [64]:
df.head()

,0,1,2,3
0,-0.276768,0.581851,2.148399,-1.279487
1,0.502277,0.856029,-0.142790,0.110079
2,-0.688065,0.433564,0.510221,-0.165131
3,-1.351779,0.546631,1.230655,1.076446
4,-1.210625,-0.306677,-1.057419,0.402057


In [65]:
df.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.061104,-0.022490,-0.013216,-0.030383
std,0.992495,0.995622,1.009888,0.972399
min,-3.000954,-4.315676,-2.857629,-3.198853
25%,-0.604967,-0.657447,-0.679803,-0.676489
50%,0.069512,-0.024156,-0.027806,-0.036945
75%,0.695465,0.641830,0.654104,0.603104
max,3.082104,3.849015,3.326698,3.617443


In [66]:
col = df[0]
col[np.abs(col)>3]

334    3.082104
860    3.039093
941   -3.000954
Name: 0, dtype: float64

In [67]:
df[(np.abs(df)>3).any(1)]

,0,1,2,3
224,0.041149,-4.315676,0.522322,-0.350425
246,0.754269,0.909113,3.326698,1.834776
250,0.842526,3.119452,1.712764,1.153649
258,1.472948,-1.062771,0.105683,-3.198853
334,3.082104,0.425523,0.594521,-0.988912
411,0.965478,-3.990082,-1.560860,0.307905
531,0.629704,3.849015,0.083691,0.493092
807,-0.739612,-0.108701,0.572723,-3.183908
860,3.039093,1.386170,-0.354378,-0.979363
939,-1.365940,1.277792,-0.658003,3.617443


# replace操作

In [69]:
ser1 = Series([1,2,3,4,1],index=['a','b','c','d','e'])
ser1

a    1
b    2
c    3
d    4
e    1
dtype: int64

In [70]:
ser1.replace(1,np.nan)

a    NaN
b    2.0
c    3.0
d    4.0
e    NaN
dtype: float64

In [71]:
ser1

a    1
b    2
c    3
d    4
e    1
dtype: int64

In [74]:
ser1.replace([1,4],[100,400],inplace=True)

In [75]:
ser1

a    100
b      2
c      3
d    400
e    100
dtype: int64

In [76]:
ser1.replace({400:4},inplace=True)

In [77]:
ser1

a    100
b      2
c      3
d      4
e    100
dtype: int64